In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from datetime import datetime
from datetime import timedelta
estfile="../estaciones.csv"
edf=pd.read_csv(estfile)
isfilelist=[]
for isfile in os.listdir("../Datos_Insitu/"):
    if isfile.endswith(".csv") and not(isfile.startswith(".")):
        isfilelist.append(isfile)
for isfile in isfilelist:
    filename="../Datos_Insitu/"+isfile
    df=pd.read_csv(filename,skiprows=2)
    df=df.drop(df.index[[0]])
    df=df.drop(df.index[range(len(df)-8,len(df))])
    filter=(df.PM25=='Sin Data')
    df1=df[~filter]
    filter2=(df1.PM25=='InVld')
    df1=df1[~filter2]
    filter3=(df1.PM25=='Apagado')
    df1=df1[~filter3]
    filter4=(df1.PM25=='RS232')
    df1=df1[~filter4]
    print "Hay %s datos invalidos de %s datos en %s" %(filter.sum()+filter2.sum()+filter3.sum()+filter4.sum(),len(df),isfile)
    satfilelist=[]
    pm25list=[]
    datelist=[]
    aodlist=[]
    distlist=[]
    estcerc=[]
    estmed=[]
    cando=True
    for satfile in os.listdir("."):
        if satfile.endswith(".csv"):
            if satfile[10:14]==isfile[-8:-4]:
                satfilelist.append(satfile)
            else:
                print "No hay archivos de MODIS para el %s" % isfile[-8:-4]
                cando=False
    if cando :
        for satfile in satfilelist:
            success=False
            sdf=pd.read_csv(satfile)
            satyear=satfile[10:14]
            satyear=int(satyear)
            satday=satfile[14:17]
            satday=int(satday)
            sathour=satfile[18:20]
            sathour=int(sathour)
            satmin=satfile[20:22]
            satmin=int(satmin)

            start=datetime(satyear,01,01)
            satdate=start+timedelta(days=satday-1,hours=sathour,minutes=satmin)
            dlist=[]
            pmlist=[]

            trange=timedelta(minutes=30)
            for i in range(len(df1)):
                date=(df1["Fecha & Hora"].iloc[i])
                day=int(date[0:2])
                month=int(date[3:5])
                year=int(date[6:10])
                hour=int(date[11:13])
                minute=int(date[14:16])
                if hour < 24 :
                    isdate=datetime(year,month,day,hour,minute)
                if satdate-trange <= isdate <=satdate+trange:
                    dlist.append(isdate)
                    try:
                        pmlist.append(int(df1.PM25.iloc[i]))
                    except:
                        print "Valor invalido en pm25, posiblemente %s pero puede haber mas" % np.unique(df1.PM25)[-1]
                        print "Hay que limpiar %s" %isfile
                    if isdate==satdate :
                        pmval=int(df1.PM25.iloc[i])
                        dtval=satdate
                        success=True
                if isdate > satdate+trange:
                    break
            pmdf=pd.DataFrame({'Date' : dlist, 'PM25': pmlist})
            if len(pmdf) > 0 :
                stname=isfile.replace("%s.csv"%year,"").upper()
                for i in range(len(edf)):
                    if edf.ESTACION.iloc[i]==stname:

                        lat=edf.LATITUD.iloc[i]
                        lon=edf.LONGITUD.iloc[i]        
                distance=np.sqrt((sdf.Latitude-lat)**2+(sdf.Longitude-lon)**2)*111.325
                ixcl=np.argmin(distance)   
                slat=sdf.Latitude.iloc[ixcl]
                slon=sdf.Longitude.iloc[ixcl]
                ixce=np.argmin((edf.LATITUD-slat)**2+(edf.LONGITUD-slon)**2)
                print "Para la estacion %s, el satelite pasa a %.2f km" %(stname,distance[ixcl])
                if edf.ESTACION.iloc[ixce]!=stname :
                    print "Sin embargo, la estacion mas cercana al punto por donde paso el satelite es %s" % edf.ESTACION.iloc[ixce]
            aodlist.append(sdf.Optical_Depth_Land_And_Ocean.iloc[ixcl])    
            distlist.append(distance[ixcl])
            estcerc.append(edf.ESTACION.iloc[ixce])
            estmed.append(stname)
            print "ODLAO =",sdf.Optical_Depth_Land_And_Ocean.iloc[ixcl],", PM25 = ", pmdf.PM25.iloc[1]
            if success:
                pm25list.append(pmval)
                datelist.append(satdate)
            else:
                pm25list.append(pmdf.PM25.iloc[0])
                datelist.append(pmdf.Date.iloc[0])
        pd.DataFrame({'Date' : datelist,'PM25' : pm25list, 'AOD' : aodlist, 'Dist' : distlist, 'EstCerc': estcerc, 'EstMed': estmed}).to_csv("../data%s.csv"%year)

Hay 1649 datos invalidos de 52560 datos en Kennedy2013.csv
Para la estacion KENNEDY, el satelite pasa a 9.57 km
ODLAO = 0.13200000627 , PM25 =  15
Hay 3742 datos invalidos de 52704 datos en Kennedy2012.csv
No hay archivos de MODIS para el 2012


Hay 1649 datos invalidos de 52560 datos en Kennedy2013.csv
Para la estacion KENNEDY, el satelite pasa a 9.57 km
ODLAO = 0.13200000627 , PM25 =  15
Hay 3742 datos invalidos de 52704 datos en Kennedy2012.csv
No hay archivos de MODIS para el 2012


In [66]:
estcerc

['KENNEDY']

In [49]:
rm file.csv

In [15]:
# avisar si no sale la hora exacta en in situ ^^  

In [18]:
np.sort(distance)

array([  9.5739337 ,  14.29022663,  14.64680363,  14.65502672,
        15.14549803,  15.76593237,  16.97915142,  17.41244013,
        18.80978832,  18.94774995,  19.24709844,  19.45899383,
        19.81310982,  19.83937856,  20.18463825,  20.92969191,
        21.26216694,  22.12865247,  22.97719574,  23.15797726,
        23.28637579,  23.31221745,  23.48282189,  23.76952002,
        24.30181841,  24.66474425,  24.70329387,  25.30303363,
        25.34654196,  25.79309008,  25.83960186,  25.91293416,
        25.94139219,  26.45685192,  26.45814483,  26.72675397,
        26.96618758,  27.0144287 ,  27.31191381,  27.57940349,
        27.62105149,  27.66732751,  28.25914311,  28.36357019,
        28.55420631,  28.71694953,  28.74107487,  28.8419333 ,
        28.90304889,  28.95473438,  29.20050593,  29.47939143,
        29.84548293,  30.28376232,  30.53664106,  30.6293391 ,
        30.81149914,  31.00937991,  31.53120192,  31.7662814 ,
        31.7892639 ,  31.87019026,  32.06252444,  32.23

In [125]:
sdf.Optical_Depth_Land_And_Ocean.iloc[ixcl]

0.13200000627

In [29]:
np.unique(df1.PM25)[-1]

'99'

In [6]:
myrange=len(df1)
myrange=10
for i in range(myrange):
    print type(df1.PM25.iloc[i]) is str

True
True
True
True
True
True
True
True
True
True


In [350]:
pm25df.PM25

0    15
1    15
2    24
3    24
4    24
5    24
6    24
Name: PM25, dtype: object